In [72]:
import pandas as pd
import os

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import Flatten, Input, Conv1D, MaxPooling1D, Dense
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical

train = pd.read_csv(os.path.join('data', 'emorynlp_train_final.csv'))
val = pd.read_csv(os.path.join('data', 'emorynlp_dev_final.csv'))
test = pd.read_csv(os.path.join('data', 'emorynlp_test_final.csv'))

train.head()

,Utterance,Speaker,Emotion,Scene_ID,Utterance_ID,Season,Episode,Start_Time,End_Time
0,"What you guys don't understand is, for us, kis...",['Monica Geller'],Joyful,1,1,1,2,00:00:02.877,00:00:07.548
1,"Yeah, right!.......Y'serious?",['Joey Tribbiani'],Neutral,1,2,1,2,00:00:04.504,00:00:07.548
2,"Oh, yeah!",['Phoebe Buffay'],Joyful,1,3,1,2,00:00:07.924,00:00:09.508
3,Everything you need to know is in that first k...,['Rachel Green'],Powerful,1,4,1,2,00:00:11.970,00:00:17.683
4,Absolutely.,['Monica Geller'],Powerful,1,5,1,2,00:00:14.139,00:00:15.097


In [106]:
# labels = LabelEncoder().fit_transform(train['Emotion'])
# n_classes = max(labels) + 1
# labels
# # labels = to_categorical(labels, n_classes)
# # labels

array([0, 2, 0, ..., 3, 6, 6])

In [110]:
labels = train['Emotion'].astype('category').cat.codes
labels = to_categorical(labels)
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [111]:
# integer encode the documents
vocab_size = 500
encoded_docs = [one_hot(d, vocab_size) for d in train['Utterance']]

In [112]:
# pad documents to a max length of 4 words
max_length = 33
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [114]:
embeddings_index = {}
f = open(os.path.join('data', 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

FileNotFoundError: [Errno 2] No such file or directory: 'data/glove.6B.100d.txt'

In [97]:

# define the model
model = Sequential()
model.add(Embedding(vocab_size, 125, input_length=max_length))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))



In [98]:
# sequence_input = Input(shape=(max_length,), dtype='int32')
# embedded_sequences = Embedding(vocab_size, 128, input_length=max_length)(sequence_input)
# x = Conv1D(128, 5, activation='relu')(embedded_sequences)
# x = MaxPooling1D(5)(x)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPooling1D(5)(x)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPooling1D(35)(x)  # global max pooling
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# preds = Dense(n_classes, activation='softmax')(x)

# model = Model(sequence_input, preds)

In [99]:

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=1, validation_split=0.1)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_39 (Embedding)     (None, 32, 125)           62500     
_________________________________________________________________
conv1d_48 (Conv1D)           (None, 30, 128)           48128     
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 28, 128)           49280     
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 14, 128)           0         
_________________________________________________________________
conv1d_50 (Conv1D)           (None, 12, 128)           49280     
_________________________________________________________________
conv1d_51 (Conv1D)           (None, 10, 128)           49280     
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 5, 128)            0         
__________

KeyboardInterrupt: 